# Box control in satellite Formation Flying 
Here we consider a system of two Earth orbiting satellites, a Chief and a Deputy, and their relative dynamics. Both satellites are subject to perturbative accelerations indicated with the vector $\mathbf d$. Eventually, we use *heyoka.py* *terminal events* machinery to study
in an innovative way the problem of controlling the satellite formation, i.e. keeping the Deputy within a predefined control box.

To refer to the deputy we use a subscript D, when no subscript is present we refer to the Chief satellite (although some time we explicitly use the subscript C for clarity).

We use a system of equations where the Deputy state ($x,y,z$) is represented in the Local Horizontal Local Vertical LHLV frame attached to the Chief satellite, whose state is instead given in the inertial frame $\left(X,Y,Z\right)$. This way the event equation takes a simple form. Vector components in this frame will be indicated with the subscripts $x,y,z$, so that for example the disturbance acceleration vector acting on the Chief will be $\left[d_{C,x}, d_{C,y}, d_{C,z}\right]$.

The LHLV frame is defined as:

$$
\left\{
\begin{array}{ll}
\mathbf {\hat i_r} &= \frac{\mathbf r}{r} \\
\mathbf {\hat i_\theta} &= \mathbf {\hat i_h} \times \mathbf {\hat i_r}\\ 
\mathbf {\hat i_h} &= \frac{\mathbf r \times \mathbf v}{\vert\mathbf r \times \mathbf v\vert}
\end{array}
\right.
$$

and has, with respect to the inertial frame, the following angular velocity:

$$
\boldsymbol \omega = \left[\omega_x, \omega_y, \omega_z\right] = \left[\frac rh d_{C,z}, 0, \frac{h}{r^2}\right]
$$

with derivative:

$$
\begin{array}{ll}
\dot \omega_x &= \frac 1h \left(\dot r d_{C,z}+ r \dot d_{C,z} - \omega_x rd_{C,y}\right) \\
\dot \omega_z &= \frac 1r\left(d_{C,y} - 2\omega_z\dot r\right)
\end{array}
$$

For the Deputy (LHLV frame), the equations of motion (EOM) take the form:
$$
(1) \qquad 
\left\{
\begin{array}{ll}
\ddot x &= \left(\omega_z - \frac \mu{r_D^3}\right)x +\dot\omega_z y-\omega_x\omega_z z + 2 \omega_z\dot y + \mu\left(\frac{1}{r^2}-\frac{r}{r_D^3}\right) + \Delta d_x \\
\ddot y &= -\dot \omega_z x + \left(\omega_x^2+\omega_z^2-\frac \mu{r^3_D}\right)y +\dot\omega_x z-2\omega_z\dot x + 2\omega_x\dot z + \Delta d_y \\ 
\ddot z &= -\omega_x\omega_z x-\dot\omega_x y + \left(\omega_x^2-\frac\mu{r^3_D}\right)z-2\omega_x\dot y + \Delta d_z
\end{array}
\right.
$$

where $r = \sqrt{X^2+Y^2+Z^2}$, $r_D = \sqrt{(r+x)^2+y^2+z^2}$, $h = \vert\mathbf r_C \times \mathbf v_C\vert$ and $\Delta \mathbf d =  \mathbf d_D- \mathbf d_C$.

For the Chief (inertial frame), the equations of motion (EOM) take the form:
$$
(2) \qquad 
\left\{
\begin{array}{ll}
\ddot X &= -\frac \mu{r^3}X + d_{C,X}\\
\ddot Y &= -\frac \mu{r^3}Y + d_{C,Y}\\ 
\ddot Z &= -\frac \mu{r^3}Z + d_{C,Z}
\end{array}
\right.
$$
Equations (1) and (2) form a generic framework to describe satellite formation flight under the influence of any perturbation. Their advantage, with respect to other set of equations commonly used when dealing with saetllite formation flying, is in their mixed representation of the Chief orbit in the inertial frame and of the deputy orbit in the LHLV frame, allowing to clearly factor the coupling terms between the two and, in our case, to write simple equations for the events triggering impulsive manouvres on the Deputy.

Before we can implement the equations in *heyoka.py* we must make a last step and define the perturbative acceleration affecting the dynamics. We will need, in particular, to compute the perturbing acceleration acting on the Chief in the inertial frame: $d_{C,X}, d_{C,Y}, d_{C,Z}$ and in the LHLV frame $d_{C,x}, d_{C,y}, d_{C,z}$ as well as $\dot d_{C,z}$.
For the Deputy we will need only to compute the perturbing acceleration in the LHLV frame $d_{D,x}, d_{D,y}, d_{D,z}$.

### The J2 perturbing acceleration
We will here consider an orbital environment where the only perturbing acceleration is that due to the $J_2$ term of the geopotential field. Its representation in cartesian coordinates can be written as:

$$
\left\{
\begin{array}{ll}
d_{C,X}&=c\frac 1{r^5} \left(5\frac{Z^2}{r^2}-1 \right) X\\
d_{C,Y}&=c\frac 1{r^5} \left(5\frac{Z^2}{r^2}-1 \right) Y\\
d_{C,Z}&=c\frac 1{r^5} \left(5\frac{Z^2}{r^2}-1 \right) Z - 2c\frac 1{r^5} Z\\
\end{array}
\right.
$$

where $c = \frac 32 \mu  J_2 R_e^2$. Immediately we also get $d_{C,x} = \mathbf d_C \cdot \mathbf {\hat i}_r$, $d_{C,y} = \mathbf d_C \cdot \mathbf {\hat i}_\theta$, $d_{C,z} = \mathbf d_C \cdot \mathbf {\hat i}_h$ and similar expressions for the Deputy and thus the differential accelerations $\Delta \mathbf d$. Expanding the previous dot products:

$$
\begin{array}{ll}
d_{C,x} = \frac{c}{r^4}\left[5Z^2-2\frac{Z^2}{r^2}-1\right] \\
d_{C,z} = 2 \frac c{r^5} Z \left( X \dot Y - Y \dot X\right) \\
\end{array}
$$

In [6]:
import heyoka as hy
import numpy as np

Let us start with defining the ODEs in an inertail frame centered at the Earth:

In [3]:
# The state (using km as unit length)
xC, yC, zC, vxC, vyC, vzC, xD, yD, zD, vxD, vyD, vzD, = hy.make_vars("xC", "yC", "zC", "vxC", "vyC", "vzC", "xD", "yD", "zD", "vxD", "vyD", "vzD")

# Parameters
mu = 398600.4418 #km^3/sec^2 
J2 = 1082.645E-6
Re = 6371 #km
c = 3./2. * J2*mu*Re**2

# Auxiliary variables
rC2 = xC**2+yC**2+zC**2
rD2 = xD**2+yD**2+zD**2

# EOM
# Chief
dxC = vxC
dyC = vyC
dzC = vzC
dvxC = (- mu * xC/rC2 + c * xC / rC2**2 * (1 - 5*zC**2/rC2)) / hy.sqrt(rC2)
dvyC = (- mu * yC/rC2 + c * yC / rC2**2 * (1 - 5*zC**2/rC2)) / hy.sqrt(rC2)
dvzC = (- mu * zC/rC2 + c * zC / rC2**2 * (3 - 5*zC**2/rC2)) / hy.sqrt(rC2)

# Deputy
dxD = vxD
dyD = vyD
dzD = vzD
dvxD = (- mu * xD/rD2 + c * xD / rD2**2 * (1 - 5*zD**2/rD2)) / hy.sqrt(rD2)
dvyD = (- mu * yD/rD2 + c * yD / rD2**2 * (1 - 5*zD**2/rD2)) / hy.sqrt(rD2)
dvzD = (- mu * zD/rD2 + c * zD / rD2**2 * (3 - 5*zD**2/rD2)) / hy.sqrt(rD2)

# Equations
eqns = [(xC, dxC), (yC, dyC), (zC, dzC), (vxC, dvxC), (vyC, dvyC), (vzC, dvzC), (xD, dxD), (yD, dyD), (zD, dzD), (vxD, dvxD), (vyD, dvyD), (vzD, dvzD)]

A convenient and most used frame to study the Deputy dynamics is the Local Horizontal Local Vertical (LHLV) frame attached to the Chief satellite. This is defined as:


in order to represent the formation in both frames the following helper functions are useful:

In [1]:
def to_relative(state):
    """Transforms the state of the Deputy to the LHLV frame
    
    Args:
        state (np.array): the system state in the inertial frame (Chief and Deputy)

    Returns:
        np.array: the system state, with the Deputy state in the LHLV frame
    """
    #In the inertial frame
    dr = np.array(state[6] - state[0], state[7] - state[1], state[8] - state[2])
    ir = dr / np.sqrt(rc2)
    ih = np.cross(state[:3], state[3:6])
    ih = ih / np.sqrt(np.cross(state[:3], state[3:6]))
    it = np.cross(ih, ir)
    # In the LHLV frame
    dr_r = np.dot(dr, ir)
    dr_t = np.dot(dr, it)
    dr_h = np.dot(dr, ih)
    
    
def to_absolute(state):
    """Transforms the state of the Deputy to the inertial frame
    
    """

In [5]:
np.cross?

Signature: np.cross(a, b, axisa=-1, axisb=-1, axisc=-1, axis=None)
Docstring:
Return the cross product of two (arrays of) vectors.

The cross product of `a` and `b` in :math:`R^3` is a vector perpendicular
to both `a` and `b`.  If `a` and `b` are arrays of vectors, the vectors
are defined by the last axis of `a` and `b` by default, and these axes
can have dimensions 2 or 3.  Where the dimension of either `a` or `b` is
2, the third component of the input vector is assumed to be zero and the
cross product calculated accordingly.  In cases where both input vectors
have dimension 2, the z-component of the cross product is returned.

Parameters
----------
a : array_like
    Components of the first vector(s).
b : array_like
    Components of the second vector(s).
axisa : int, optional
    Axis of `a` that defines the vector(s).  By default, the last axis.
axisb : int, optional
    Axis of `b` that defines the vector(s).  By default, the last axis.
axisc : int, optional
    Axis of `c` contai